# Multimodal Endpoint with LiteLLM, ComfyUI, and Trulens

This notebook is designed to serve as a backend for a multimodal system that integrates text and image generation with evaluation metrics. It leverages LiteLLM for language model completions, ComfyUI for image generation, and Trulens for response evaluation. The system is structured to handle multi-turn conversations, with each conversation's data being stored in a separate PostgreSQL table for easy tracking and analysis.

## Notebook Structure

The notebook is organized into several sections, each contained within its own cell or group of cells:

1. **Environment Setup**: Import libraries, set environment variables, and install any necessary packages.
2. **Configuration**: Define API keys, endpoints, and global variables.
3. **LiteLLM Completion Function**: Code to interact with LiteLLM for text generation.
4. **ComfyUI Image Generation Function**: Code to generate images using ComfyUI based on text prompts.
5. **File Watcher Setup**: Implement a file watcher to monitor for new images and display them.
6. **Trulens Evaluation Setup**: Configure Trulens to evaluate responses and send data to PostgreSQL.
7. **PostgreSQL Integration**: Functions for database interactions, including table creation and data insertion.
8. **Main Workflow**: The main function that orchestrates the text generation, image generation, and evaluation process.
9. **Gradio Interface (Placeholder)**: A placeholder for future integration with a Gradio front-end.
10. **Execution and Testing**: Cells to execute the main workflow with test inputs.
11. **Documentation and Notes**: Additional explanations and documentation for users.

## Purpose

The goal of this notebook is to create an endpoint that accepts text prompts, generates corresponding images, and evaluates the responses. It is designed to be part of a larger system that could include a front-end interface for user interaction. The notebook will demonstrate the backend functionality, including the integration of different APIs and tools, and the handling of multi-turn conversations with persistent storage.

Please ensure that you replace any placeholder values with your actual API keys, paths, and connection strings as needed before running the notebook.

In [11]:
# Cell 1: Environment Setup and Package Installation


# Import necessary libraries
import os
from getpass import getpass
import psycopg2
from litellm import completion
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler

# Install required packages
%pip install litellm[proxy] psycopg2-binary watchdog requests
%pip install --extra-index-url https://download.pytorch.org/whl/cu121 hordelib


# Prompt for API keys and sensitive information
os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API key: ")
os.environ["GEMINI_API_KEY"] = getpass("Enter your Gemini API key: ")
os.environ["VERTEX_PROJECT"] = getpass("Enter your Vertex project ID: ")
os.environ["VERTEX_LOCATION"] = getpass("Enter your Vertex location: ")

# Set up global variables and paths
IMAGE_OUTPUT_FOLDER = "/path/to/generated_images"  # Replace with the actual path to the image output folder
DATABASE_FILE_PATH = "path/to/your/directory/default.sqlite"  # Replace with the actual path to your SQLite database file

print("Environment setup and package installation complete.")

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com, https://download.pytorch.org/whl/cu121

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


ModuleNotFoundError: No module named 'comfyui'

In [9]:
# Cell 2: Configuration Setup

# Define the model names for LiteLLM completion
OPENAI_MODEL = "gpt-3.5-turbo"
GEMINI_MODEL = "google/gemini-model"  # Replace with the actual model name if different

# Define the path to the model cache for hordelib
os.environ["AIWORKER_CACHE_HOME"] = "/path/to/your/models"  # Replace with the actual path to your models

print("Configuration setup complete.")

Configuration setup complete.


In [ ]:
# Cell 3: LiteLLM Proxy Server Configuration

# Assuming the necessary packages are installed and environment variables are set from Cell 1
# Assuming global variables and paths are set from Cell 2

# Import necessary libraries
import yaml
from pathlib import Path

# Define the configuration for the LiteLLM proxy server
config = {
    'model_list': [
        {
            'model_name': 'gpt-3.5-turbo',
            'litellm_params': {
                'model': 'openai/gpt-3.5-turbo',
                'api_base': 'https://api.openai.com',
                'api_key': os.getenv("OPENAI_API_KEY")
            }
        },
        {
            'model_name': 'gemini-pro',
            'litellm_params': {
                'model': 'vertex_ai/gemini-pro',
                'api_base': 'https://vertex-ai.google.com',
                'api_key': os.getenv("GEMINI_API_KEY")
            }
        }
    ],
    'litellm_settings': {
        'drop_params': True,
        'set_verbose': True
    },
    'general_settings': {
        'master_key': 'sk-my_special_key'  # Replace with a master key of your choice
    }
}

# Write the config to a YAML file
config_path = Path('config.yaml')
with open(config_path, 'w') as config_file:
    yaml.dump(config, config_file, default_flow_style=False)

print(f"Config file created at: {config_path.resolve()}")

# Start the LiteLLM proxy server with the config file
!litellm --config {config_path}

In [19]:
import json
from urllib import request, parse
import random

#This is the ComfyUI api prompt format.

#If you want it for a specific workflow you can "enable dev mode options"
#in the settings of the UI (gear beside the "Queue Size: ") this will enable
#a button on the UI to save workflows in api format.

#keep in mind ComfyUI is pre alpha software so this format will change a bit.



An error occurred: HTTP Error 400: Bad Request
Request data: b'{"prompt": {"3": {"class_type": "KSampler", "inputs": {"cfg": 8, "denoise": 1, "latent_image": ["5", 0], "model": ["4", 0], "negative": ["7", 0], "positive": ["6", 0], "sampler_name": "euler", "scheduler": "normal", "seed": 8566257, "steps": 20}}, "4": {"class_type": "CheckpointLoaderSimple", "inputs": {"ckpt_name": "v1-5-pruned-emaonly.ckpt"}}, "5": {"class_type": "EmptyLatentImage", "inputs": {"batch_size": 1, "height": 512, "width": 512}}, "6": {"class_type": "CLIPTextEncode", "inputs": {"clip": ["4", 1], "text": "masterpiece best quality girl"}}, "7": {"class_type": "CLIPTextEncode", "inputs": {"clip": ["4", 1], "text": "bad hands"}}, "8": {"class_type": "VAEDecode", "inputs": {"samples": ["3", 0], "vae": ["4", 2]}}, "9": {"class_type": "SaveImage", "inputs": {"filename_prefix": "ComfyUI", "images": ["8", 0]}}}}'
Request headers: {'Content-type': 'application/json'}
An error occurred: HTTP Error 400: Bad Request
Request

In [16]:
# Cell 5: Send Data to Endpoint and Receive Image Link

import requests
import json

# Define the endpoint URL
endpoint_url = "https://5349-104-255-9-187.ngrok-free.app/prompt"
prompt = "an ancient llamia monster"
model = "LATENT"

# Define the data payload similar to the object outlined in the horde cell
data_payload = {
        "sampler_name": "k_dpmpp_2m",
        "cfg_scale": 7.5,
        "denoising_strength": 1.0,
        "seed": 123456789,  # Replace with a dynamic seed if needed
        "height": 512,
        "width": 512,
        "karras": False,
        "tiling": False,
        "hires_fix": False,
        "clip_skip": 1,
        "control_type": None,
        "image_is_control": False,
        "return_control_map": False,
        "prompt": prompt,
        "ddim_steps": 25,
        "n_iter": 1,
        "model": model,
    }

# Make the POST request to the endpoint
response = requests.post(endpoint_url, json=data_payload)

# Print the full response object for debugging
print("Status Code:", response.status_code)
print("Response Headers:", response.headers)
print("Response Text:", response.text)

# Check if the request was successful
try:
    response_data = response.json()
    print("JSON Response:", json.dumps(response_data, indent=2))
except json.JSONDecodeError:
    print("Response is not JSON. Printed 'Response Text' above should give clues as to why.")

Status Code: 500
Response Headers: {'Content-Length': '55', 'Content-Type': 'text/plain; charset=utf-8', 'Date': 'Mon, 25 Dec 2023 00:22:10 GMT', 'Ngrok-Trace-Id': '5b8625c1d2ae3268523714eb2b347528', 'Server': 'Python/3.10 aiohttp/3.9.1'}
Response Text: 500 Internal Server Error

Server got itself in trouble
Response is not JSON. Printed 'Response Text' above should give clues as to why.


In [ ]:
# Cell 4: Image Generation Function using HordeLib
import hordelib
from hordelib.horde import HordeLib
from hordelib.shared_model_manager import SharedModelManager

# Initialize hordelib (Note: This will erase all command line arguments from argv)
hordelib.initialise()
# Assuming the necessary models are loaded and available in AIWORKER_CACHE_HOME
def generate_image_with_hordelib(prompt, output_folder, model_name="Deliberate"):
    generate = HordeLib()
    SharedModelManager.loadModelManagers(compvis=True)
    SharedModelManager.manager.load(model_name)

    data = {
        "sampler_name": "k_dpmpp_2m",
        "cfg_scale": 7.5,
        "denoising_strength": 1.0,
        "seed": 123456789,  # Replace with a dynamic seed if needed
        "height": 512,
        "width": 512,
        "karras": False,
        "tiling": False,
        "hires_fix": False,
        "clip_skip": 1,
        "control_type": None,
        "image_is_control": False,
        "return_control_map": False,
        "prompt": prompt,
        "ddim_steps": 25,
        "n_iter": 1,
        "model": model_name,
    }
    pil_image = generate.basic_inference_single_image(data).image
    image_path = os.path.join(output_folder, "generated_image.png")
    pil_image.save(image_path)
    return image_path

# Example usage
prompt = "an ancient llamia monster"
image_path = generate_image_with_hordelib(prompt, IMAGE_OUTPUT_FOLDER)

In [ ]:
# Cell 7: Trulens and SQLite Integration

import sqlite3
from pathlib import Path

# Set up the path for the SQLite database
DATABASE_FILE_PATH = Path("path/to/your/directory/default.sqlite")  # Replace with the actual path to your directory

# Function to create a new table in SQLite for each conversation
def create_conversation_table(conversation_id):
    # Connect to the SQLite database
    conn = sqlite3.connect(DATABASE_FILE_PATH)
    cur = conn.cursor()
    
    # Create a table for the conversation
    table_name = f"conversation_{conversation_id}"
    cur.execute(f"CREATE TABLE IF NOT EXISTS {table_name} (id INTEGER PRIMARY KEY, data TEXT);")
    
    # Commit changes and close the connection
    conn.commit()
    cur.close()
    conn.close()

# Function to insert data into the conversation table
def insert_into_conversation_table(conversation_id, data):
    # Connect to the SQLite database
    conn = sqlite3.connect(DATABASE_FILE_PATH)
    cur = conn.cursor()
    
    # Insert data into the table
    table_name = f"conversation_{conversation_id}"
    cur.execute(f"INSERT INTO {table_name} (data) VALUES (?);", (data,))
    
    # Commit changes and close the connection
    conn.commit()
    cur.close()
    conn.close()

# Example usage
conversation_id = "12345"  # Unique identifier for the conversation
create_conversation_table(conversation_id)
data_entry = json.dumps({'text_response': 'Example response', 'image_path': 'path/to/image.png'})
insert_into_conversation_table(conversation_id, data_entry)